In [ ]:
import openai as client

assistant = client.beta.assistants.create(
    name="Book Assistant",
    instructions="You help users with their question on the files they upload.",
    model="gpt-4-1106-preview",
    tools=[{"type": "file_search"}],
)
assistant

In [1]:
import openai as client

# assistant = client.beta.assistants.create(
#    name="Book Assistant",
#    instructions="You help users with their question on the files they upload.",
#    model="gpt-4-1106-preview",
#    tools=[{"type": "file_search"}],
#)
assistant_id = "asst_kjnv9pKth1asyF06vZFrGAWN"

In [2]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want you to help me with this file",
        }
    ]
)
thread

Thread(id='thread_szLUojUKpLeU66zky1f1qrpR', created_at=1722843350, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [3]:
file = client.files.create(
    file=client.file_from_path("C:\Python\\fullstack_gpt\\files\chapter_one.txt"), purpose="assistants"
)
file

FileObject(id='file-rcldzVm3dcFTptUwk2U4MjKS', bytes=15801, created_at=1722843354, filename='chapter_one.txt', object='file', purpose='assistants', status='processed', status_details=None)

In [4]:
client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Please refer to the uploaded file for my question. ",
  attachments=[{
    "file_id": file.id,
    "tools": [{"type": "file_search",}],}],
)


Message(id='msg_qZ0RSzKnDAmCTFgcc3URsNcM', assistant_id=None, attachments=[Attachment(file_id='file-rcldzVm3dcFTptUwk2U4MjKS', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please refer to the uploaded file for my question. '), type='text')], created_at=1722843364, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_szLUojUKpLeU66zky1f1qrpR')

In [11]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_21FFoiobdUUbQnTiS6avIzUX', assistant_id='asst_kjnv9pKth1asyF06vZFrGAWN', cancelled_at=None, completed_at=None, created_at=1722843406, expires_at=1722844006, failed_at=None, incomplete_details=None, instructions='You help users with their question on the files they upload.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_szLUojUKpLeU66zky1f1qrpR', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [6]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source: {annotation.file_citation}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [13]:
get_run(run.id, thread.id).status

'completed'

In [14]:
get_messages(thread.id)

user: I want you to help me with this file
user: Please refer to the uploaded file for my question. 
assistant: It appears that your question wasn't included in your initial message, and since you've asked me to refer to the uploaded file, I'm not able to see any specific question or context. Could you please provide me with more details or clarify what aspect of the file you need assistance with? Once I have a better understanding of your question, I can provide you with the relevant information from the file.
user: Who is Mr. Goldwin Smith?
assistant: Mr. Goldwin Smith is mentioned in the context of his observation about Jane Austen's art. He has been quoted as saying that "metaphor has been exhausted in depicting the perfection of it [Jane Austen's art], combined with the narrowness of her field," and he adds that one need not go beyond Austen's own comparison to the art of a miniature painter. This observation is used to emphasize the quality of Jane Austen's work and her craftsman

In [ ]:
send_message(
    thread.id,
    "Where does he work?",
)

In [8]:
send_message(
    thread.id,
    "Who is Mr. Goldwin Smith?",
)

Message(id='msg_z89xCprb7mdsw1JIyrEJdbCM', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who is Mr. Goldwin Smith?'), type='text')], created_at=1722843381, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_szLUojUKpLeU66zky1f1qrpR')

In [1]:
import os
import boto3

session = boto3.Session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_KEY"),
)

client = session.client("bedrock-runtime", "us-east-1")

In [2]:
from langchain.chat_models import BedrockChat
from langchain.prompts import ChatPromptTemplate

chat = BedrockChat(
    client=client,
    model_id="anthropic.claude-v2",
    model_kwargs={
        "temperature": 0.1,
    },
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Translate this sentence from {lang_a} to {lang_b}: {sentence}",
        ),
    ]
)

chain = prompt | chat

chain.invoke(
    {
        "lang_a": "English",
        "lang_b": "Korean",
        "sentence": "I love amazon!",
    }
)

AIMessage(content=' Here is the translation:\n\n나는 아마존을 좋아해!', id='run-5305f0ab-f179-4754-8ff1-381a3a2f3193-0')